In [2]:
from montecarlo import MonteCarlo
from scipy.stats import norm
import numpy as np

Analytical value of $\delta$: 

In [2]:
norm.cdf(1 / (0.2 * np.sqrt(1)) * (np.log(100 / 99) + (0.06 + ((0.2 ** 2) / 2)) * 1)) - 1

-0.3262644882651039

## Bump and evaluate
Different seed:

In [13]:
n = 100
for s in [100, 1000, 10000]:
    v_bumped = np.empty((n, s))
    v_unbumped = np.empty((n, s))
    for e in [1, 0.1, 0.01]:
        for i in range(n):
            v_bumped[i] = MonteCarlo(100+e, 0.06, 1, 99, 365, 0.2).run_immediately(s)
            v_unbumped[i] = MonteCarlo(100, 0.06, 1, 99, 365, 0.2).run_immediately(s)
        deltas = (np.mean(v_bumped, axis=1) - np.mean(v_unbumped, axis=1)) / e
        print(f'e={e}, s={s}: delta={np.mean(deltas)} ({np.std(deltas) / np.sqrt(n)})')

e=1, s=100: delta=-0.3240071879587982 (0.10910886879088744)
e=0.1, s=100: delta=-0.41206823215631083 (1.1226667952542577)
e=0.01, s=100: delta=-3.9787113113312045 (11.225319142999236)
e=1, s=1000: delta=-0.3177095057072285 (0.034881161890268686)
e=0.1, s=1000: delta=-0.14527005585360514 (0.38397495697741596)
e=0.01, s=1000: delta=-7.059835984847976 (3.8177325046815134)
e=1, s=10000: delta=-0.31853159119042845 (0.011472697540070163)
e=0.1, s=10000: delta=-0.39104712718909534 (0.10676153235219894)
e=0.01, s=10000: delta=-0.6356212003561543 (1.220446205779881)


Same seed:

In [14]:
n = 100
for s in [100, 1000, 10000]:
    v_bumped = np.empty((n, s))
    v_unbumped = np.empty((n, s))
    for e in [1, 0.1, 0.01]:
        for i in range(n):
            v_bumped[i] = MonteCarlo(100+e, 0.06, 1, 99, 365, 0.2).run_immediately(s, seed=i)
            v_unbumped[i] = MonteCarlo(100, 0.06, 1, 99, 365, 0.2).run_immediately(s, seed=i)
        deltas = (np.mean(v_bumped, axis=1) - np.mean(v_unbumped, axis=1)) / e
        print(f'e={e}, s={s}: delta={np.mean(deltas)} ({np.std(deltas) / np.sqrt(n)})')

e=1, s=100: delta=-0.31700541743583693 (0.003808907095506057)
e=0.1, s=100: delta=-0.3241439763237034 (0.00379231100903476)
e=0.01, s=100: delta=-0.3248974235110383 (0.003768353982781734)
e=1, s=1000: delta=-0.31715142278333147 (0.0012352933070031151)
e=0.1, s=1000: delta=-0.3253099948341837 (0.0012585618063543451)
e=0.01, s=1000: delta=-0.3261716624976118 (0.001259537209706986)
e=1, s=10000: delta=-0.3171257583332522 (0.00038409145051077954)
e=0.1, s=10000: delta=-0.32509618464485385 (0.0003934633763229457)
e=0.01, s=10000: delta=-0.3259315658377204 (0.00039021611896200066)


## Delta digital option
Using bump and evaluate:

In [4]:
 n = 100
for s in [100, 1000, 10000]:
    v_bumped = np.empty((n, s))
    v_unbumped = np.empty((n, s))
    for e in [1, 0.1, 0.01]:
        for i in range(n):
            v_bumped[i] = MonteCarlo(100+e, 0.06, 1, 99, 365, 0.2, digital=True).run_immediately(s, seed=i)
            v_unbumped[i] = MonteCarlo(100, 0.06, 1, 99, 365, 0.2, digital=True).run_immediately(s, seed=i)
        deltas = (np.mean(v_bumped, axis=1) - np.mean(v_unbumped, axis=1)) / e
        print(f'e={e}, s={s}: delta={np.mean(deltas)} ({np.std(deltas) / np.sqrt(n)})')

e=1, s=100: delta=-0.016009997070932223 (0.001172487388807058)
e=0.1, s=100: delta=-0.01695176160451647 (0.003618141186568765)
e=0.01, s=100: delta=-0.02825293600752743 (0.01606529947074415)
e=1, s=1000: delta=-0.018223143724855215 (0.00037181813171668633)
e=0.1, s=1000: delta=-0.01921199648511868 (0.0012054586029878415)
e=0.01, s=1000: delta=-0.01789352613810047 (0.004355597334623524)
e=1, s=10000: delta=-0.017864331437559614 (0.00012668532354368113)
e=0.1, s=10000: delta=-0.018694025991647358 (0.0003851747863657052)
e=0.01, s=10000: delta=-0.01723429096459156 (0.0011226765485297355)


Using Likelihood ratio:

In [12]:
n = 100
for s in [100, 1000, 10000]:
    v = np.empty((n, s))
    deltas = np.empty(n)
    for e in [1, 0.1, 0.01]:
        for i in range(n):
            v[i] = MonteCarlo(100, 0.06, 1, 99, 365, 0.2, digital=True).run_immediately(s, seed=i)
            
            np.random.seed(i)
            epsilons = np.random.normal(size=s)
            deltas[i] = np.mean(v[i] * epsilons / (100 * 0.2 * np.sqrt(1)))
        
        print(f'e={e}, s={s}: delta={np.mean(deltas)} ({np.std(deltas) / np.sqrt(n)})')

e=1, s=100: delta=-0.018085876561628328 (0.0002692332236875202)
e=0.1, s=100: delta=-0.018085876561628328 (0.0002692332236875202)
e=0.01, s=100: delta=-0.018085876561628328 (0.0002692332236875202)
e=1, s=1000: delta=-0.01815522588671591 (8.735263140284292e-05)
e=0.1, s=1000: delta=-0.01815522588671591 (8.735263140284292e-05)
e=0.01, s=1000: delta=-0.01815522588671591 (8.735263140284292e-05)
e=1, s=10000: delta=-0.018200035014475827 (2.594506368620828e-05)
e=0.1, s=10000: delta=-0.018200035014475827 (2.594506368620828e-05)
e=0.01, s=10000: delta=-0.018200035014475827 (2.594506368620828e-05)


Using pathwise (work in progress):

In [13]:
n = 100
for s in [100, 1000, 10000]:
    v = np.empty((n, s))
    deltas = np.empty(n)
    for e in [1, 0.1, 0.01]:
        for i in range(n):
            stock_prices = np.array([100] * s, dtype=np.float64)
            stock_prices *= np.exp((0.06 - 0.5 * 0.2**2) + 0.2 *  np.random.normal(size=s))
            payoffs = norm.pdf(stock_prices, 99) * np.exp(-0.06)

            deltas[i] = np.mean(payoffs * stock_prices / 100)
        
        print(f'e={e}, s={s}: delta={np.mean(deltas)} ({np.std(deltas) / np.sqrt(n)})')

e=1, s=100: delta=0.019304483116354982 (0.0006704802191635525)
e=0.1, s=100: delta=0.018716824465667838 (0.0006493838371922119)
e=0.01, s=100: delta=0.019824779314216102 (0.0006437952614560185)
e=1, s=1000: delta=0.018381499447338915 (0.00022138502930841463)
e=0.1, s=1000: delta=0.018046914776424793 (0.0001975933737369344)
e=0.01, s=1000: delta=0.01828641470513096 (0.00018408536617889074)
e=1, s=10000: delta=0.01830617812488203 (5.9860069597907725e-05)
e=0.1, s=10000: delta=0.018131772408621166 (7.261560477348804e-05)
e=0.01, s=10000: delta=0.018162220632919036 (6.455600101196671e-05)
